In [1]:
from pyspark import broadcast
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
# perfOld = spark.read.option("header",True).csv("/local/saralihalli/HOME/rapidsMortgage/output/scalaSparkSubmit/perf/")
# acqOld = spark.read.option("header",True).csv("/local/saralihalli/HOME/rapidsMortgage/output/scalaSparkSubmit/acq/")


perfOld = spark.read.parquet("/local/saralihalli/HOME/rapidsMortgage/output/scalaSparkSubmit/perf/")
acqOld = spark.read.parquet("/local/saralihalli/HOME/rapidsMortgage/output/scalaSparkSubmit/acq/")


In [3]:
perfNew = spark.read.parquet("/local/saralihalli/HOME/output/scalaSparkSubmit/perf/")
acqNew = spark.read.parquet("/local/saralihalli/HOME/output/scalaSparkSubmit/acq/")


In [4]:
perfOld.dtypes == perfNew.dtypes

True

In [5]:
perfOld.columns == perfNew.columns

True

In [6]:
for col in perfNew.columns:
    perfOld = perfOld.withColumnRenamed(col, col + "_old")

In [7]:
perfOld

DataFrame[loan_id_old: bigint, monthly_reporting_period_old: string, servicer_old: string, interest_rate_old: double, current_actual_upb_old: double, loan_age_old: double, remaining_months_to_legal_maturity_old: double, adj_remaining_months_to_maturity_old: double, maturity_date_old: string, msa_old: double, current_loan_delinquency_status_old: int, mod_flag_old: string, zero_balance_code_old: string, zero_balance_effective_date_old: string, last_paid_installment_date_old: string, foreclosed_after_old: string, disposition_date_old: string, foreclosure_costs_old: double, prop_preservation_and_repair_costs_old: double, asset_recovery_costs_old: double, misc_holding_expenses_old: double, holding_taxes_old: double, net_sale_proceeds_old: double, credit_enhancement_proceeds_old: double, repurchase_make_whole_proceeds_old: string, other_foreclosure_proceeds_old: double, non_interest_bearing_upb_old: double, principal_forgiveness_upb_old: string, repurchase_make_whole_proceeds_flag_old: strin

In [8]:
perf = perfOld.join(perfNew, [perfOld["loan_id_old"] == perfNew["loan_id"], perfOld["monthly_reporting_period_old"] == perfNew["monthly_reporting_period"]],"right")





In [9]:
from pyspark.sql.functions import col
for colN in perfNew.columns:
    perf = perf.withColumn(colN + "_match", when( (col(colN).isNull() & col(colN + "_old").isNull())  | (col(colN) == col(colN + "_old")), 0).otherwise(1) )

In [10]:
perf

DataFrame[loan_id_old: bigint, monthly_reporting_period_old: string, servicer_old: string, interest_rate_old: double, current_actual_upb_old: double, loan_age_old: double, remaining_months_to_legal_maturity_old: double, adj_remaining_months_to_maturity_old: double, maturity_date_old: string, msa_old: double, current_loan_delinquency_status_old: int, mod_flag_old: string, zero_balance_code_old: string, zero_balance_effective_date_old: string, last_paid_installment_date_old: string, foreclosed_after_old: string, disposition_date_old: string, foreclosure_costs_old: double, prop_preservation_and_repair_costs_old: double, asset_recovery_costs_old: double, misc_holding_expenses_old: double, holding_taxes_old: double, net_sale_proceeds_old: double, credit_enhancement_proceeds_old: double, repurchase_make_whole_proceeds_old: string, other_foreclosure_proceeds_old: double, non_interest_bearing_upb_old: double, principal_forgiveness_upb_old: string, repurchase_make_whole_proceeds_flag_old: strin

In [11]:
finalPerfColumns = ['current_loan_delinquency_status', 'current_actual_upb', 'interest_rate' , 'msa' , 'non_interest_bearing_upb']
for colN in finalPerfColumns:
    print(colN, perf.filter(col('loan_id_old').isNotNull()).select(sum(colN + '_match')).collect())
    
    
    
    
    

current_loan_delinquency_status [Row(sum(current_loan_delinquency_status_match)=244599)]
current_actual_upb [Row(sum(current_actual_upb_match)=1712714)]
interest_rate [Row(sum(interest_rate_match)=244659)]
msa [Row(sum(msa_match)=0)]
non_interest_bearing_upb [Row(sum(non_interest_bearing_upb_match)=0)]


loan_id [Row(sum(loan_id_match)=94894)]
monthly_reporting_period [Row(sum(monthly_reporting_period_match)=94894)]
servicer [Row(sum(servicer_match)=9005257)]
interest_rate [Row(sum(interest_rate_match)=338282)]
current_actual_upb [Row(sum(current_actual_upb_match)=1807608)]
loan_age [Row(sum(loan_age_match)=337884)]
remaining_months_to_legal_maturity [Row(sum(remaining_months_to_legal_maturity_match)=338177)]
adj_remaining_months_to_maturity [Row(sum(adj_remaining_months_to_maturity_match)=322512)]
maturity_date [Row(sum(maturity_date_match)=337813)]
msa [Row(sum(msa_match)=94894)]
current_loan_delinquency_status [Row(sum(current_loan_delinquency_status_match)=338778)]
mod_flag [Row(sum(mod_flag_match)=337908)]
zero_balance_code [Row(sum(zero_balance_code_match)=8851689)]
zero_balance_effective_date [Row(sum(zero_balance_effective_date_match)=1408)]
last_paid_installment_date [Row(sum(last_paid_installment_date_match)=90)]
foreclosed_after [Row(sum(foreclosed_after_match)=169)]
disposition_date [Row(sum(disposition_date_match)=195)]
foreclosure_costs [Row(sum(foreclosure_costs_match)=187)]
prop_preservation_and_repair_costs [Row(sum(prop_preservation_and_repair_costs_match)=179)]
asset_recovery_costs [Row(sum(asset_recovery_costs_match)=339)]
misc_holding_expenses [Row(sum(misc_holding_expenses_match)=157)]
holding_taxes [Row(sum(holding_taxes_match)=159)]
net_sale_proceeds [Row(sum(net_sale_proceeds_match)=115)]
credit_enhancement_proceeds [Row(sum(credit_enhancement_proceeds_match)=453)]
repurchase_make_whole_proceeds [Row(sum(repurchase_make_whole_proceeds_match)=566)]
other_foreclosure_proceeds [Row(sum(other_foreclosure_proceeds_match)=242)]
non_interest_bearing_upb [Row(sum(non_interest_bearing_upb_match)=15386)]
principal_forgiveness_upb [Row(sum(principal_forgiveness_upb_match)=7588)]
repurchase_make_whole_proceeds_flag [Row(sum(repurchase_make_whole_proceeds_flag_match)=8852008)]
foreclosure_principal_write_off_amount [Row(sum(foreclosure_principal_write_off_amount_match)=5079)]
servicing_activity_indicator [Row(sum(servicing_activity_indicator_match)=8389520)]
quarter [Row(sum(quarter_match)=94894)]


In [12]:
perfOld.groupBy(["monthly_reporting_period_old","loan_id_old"]).agg(count("*")).where("count(1) > 1").show()

+----------------------------+-----------+--------+
|monthly_reporting_period_old|loan_id_old|count(1)|
+----------------------------+-----------+--------+
+----------------------------+-----------+--------+



In [13]:
current_loan_delinquency_status [Row(sum(current_loan_delinquency_status_match)=338778)]


SyntaxError: keyword can't be an expression (<ipython-input-13-74d0e979bf23>, line 1)

In [ ]:
# perf.where(col('loan_id_old').isNotNull()).show()

In [ ]:
perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','current_loan_delinquency_status', 'current_loan_delinquency_status_old']).where('current_loan_delinquency_status_match = 1').show()
perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','current_actual_upb', 'current_actual_upb_old']).where('current_actual_upb_match = 1').show()
perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','interest_rate', 'interest_rate_old']).where('interest_rate_match = 1').show()
# perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','msa', 'msa_old']).where('msa_match = 1').show()
# perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','non_interest_bearing_upb', 'non_interest_bearing_upb_old']).where('non_interest_bearing_upb_match = 1').show()



In [ ]:
perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','current_loan_delinquency_status', 'current_loan_delinquency_status_old']).where('current_loan_delinquency_status_match = 1').where('current_loan_delinquency_status = 0').show()




In [ ]:
perf.select(['loan_id','loan_id_old','interest_rate', 'interest_rate_old', 'interest_rate_match']).where('loan_id = 803738209951').distinct().show()


In [ ]:
perf.select(['loan_id','loan_id_old','interest_rate', 'interest_rate_old']).where('loan_id = 803738209951').distinct().show()


In [40]:
# perf.filter(col('loan_id_old').isNotNull()).select(['loan_id','current_loan_delinquency_status', 'current_loan_delinquency_status_old']).where('current_loan_delinquency_status_match = 1').show()

perf.filter(col('loan_id_old').isNotNull()).where('current_actual_upb_match = 0').groupBy('current_loan_delinquency_status').count().orderBy(col('count').desc()).show(100)
perf.filter(col('loan_id_old').isNotNull()).where('current_actual_upb_match = 0').groupBy('current_loan_delinquency_status_old').count().orderBy(col('count').desc()).show(100)





+-------------------------------+-------+
|current_loan_delinquency_status|  count|
+-------------------------------+-------+
|                              0|6975787|
|                              1| 238655|
|                              2|  55980|
|                              3|  22019|
|                              4|  14608|
|                              5|  11265|
|                              6|   9269|
|                              7|   7543|
|                              8|   6191|
|                              9|   5111|
|                             10|   4324|
|                             11|   3767|
|                             12|   3194|
|                             13|   2616|
|                             14|   2214|
|                             15|   1899|
|                             16|   1720|
|                             17|   1448|
|                             18|   1306|
|                             19|   1124|
|                             20| 

In [53]:
perf.filter('current_actual_upb_match = 1').select(['loan_id','current_actual_upb', 'current_actual_upb_old']).groupBy('current_actual_upb').count().orderBy(col('count').desc()).show()


+------------------+-------+
|current_actual_upb|  count|
+------------------+-------+
|               0.0|1714010|
|          90030.94|     48|
|           25033.5|     48|
|          71420.72|     48|
|          51680.83|     48|
|          76854.25|     47|
|          25271.94|     46|
|          31133.19|     46|
|          15677.21|     41|
|          78752.51|     38|
|          51327.72|     37|
|           2733.17|     36|
|           35716.3|     36|
|          30892.68|     34|
|           7390.59|     34|
|          51309.33|     34|
|          13779.16|     30|
|          51225.44|     30|
|          42694.05|     29|
|          57977.99|     29|
+------------------+-------+
only showing top 20 rows



In [16]:
perf.filter('current_actual_upb_match = 1').select(['loan_id','current_actual_upb', 'current_actual_upb_old']).filter("current_actual_upb = 0.0").groupBy('current_actual_upb_old').count().orderBy(col('count').desc()).show()






+----------------------+-------+
|current_actual_upb_old|  count|
+----------------------+-------+
|                  null|1469726|
|              253000.0|    128|
|              240000.0|    124|
|              100000.0|    121|
|              200000.0|     75|
|              150000.0|     74|
|               50000.0|     70|
|               60000.0|     65|
|               80000.0|     65|
|               70000.0|     64|
|               85000.0|     60|
|               75000.0|     57|
|               90000.0|     56|
|              140000.0|     55|
|              110000.0|     50|
|               30000.0|     47|
|               40000.0|     46|
|              125000.0|     46|
|              120000.0|     45|
|               65000.0|     43|
+----------------------+-------+
only showing top 20 rows



In [88]:
perf.select(['loan_id','current_actual_upb', 'current_actual_upb_old']).where('current_actual_upb_match = 1').show()

+------------+------------------+----------------------+
|     loan_id|current_actual_upb|current_actual_upb_old|
+------------+------------------+----------------------+
|263055415026|               0.0|                  null|
|330414171561|               0.0|                  null|
|294214882652|               0.0|                  null|
|880810179517|               0.0|                  null|
|918451611732|               0.0|                  null|
|865382851209|               0.0|                  null|
|918135573229|               0.0|                  null|
|393422066928|               0.0|                  null|
|430593550550|               0.0|                  null|
|427943988327|               0.0|                  null|
|387233861776|               0.0|                  null|
|441755291071|               0.0|                  null|
|430419305785|               0.0|                  null|
|411039819128|               0.0|                  null|
|553930052725|               0.

In [89]:
perf.count()

9189573

In [58]:
perf.filter("interest_rate_match = 1").count()

339553

In [59]:
perfNew.count()

9189573

In [92]:
perfOld.count()

9094679

In [61]:
acqNew.dtypes == acqOld.dtypes

True

In [62]:
for col in acqNew.columns:
    acqOld = acqOld.withColumnRenamed(col, col + "_old")

In [64]:
acqOld.groupBy(["loan_id_old"]).agg(count("*")).where("count(1) > 1").show()

+-----------+--------+
|loan_id_old|count(1)|
+-----------+--------+
+-----------+--------+



In [65]:
acq = acqOld.join(acqNew, [acqOld["loan_id_old"] == acqNew["loan_id"]],"right")


In [67]:
acq.count()

246863

In [68]:
acqOld.count()

246863

In [69]:
acqNew.count()

246863

In [80]:
from pyspark.sql.functions import col
for colN in acqNew.columns:
    acq = acq.withColumn(colN + "_match", when( (col(colN).isNull() & col(colN + "_old").isNull())  | (col(colN) == col(colN + "_old")), 0).otherwise(1) )

In [81]:
for colN in acqNew.columns:
    print(colN, acq.select(sum(colN + '_match')).collect())


loan_id [Row(sum(loan_id_match)=0)]
orig_channel [Row(sum(orig_channel_match)=0)]
seller_name [Row(sum(seller_name_match)=0)]
orig_interest_rate [Row(sum(orig_interest_rate_match)=0)]
orig_upb [Row(sum(orig_upb_match)=0)]
orig_loan_term [Row(sum(orig_loan_term_match)=0)]
orig_date [Row(sum(orig_date_match)=0)]
first_pay_date [Row(sum(first_pay_date_match)=0)]
orig_ltv [Row(sum(orig_ltv_match)=0)]
orig_cltv [Row(sum(orig_cltv_match)=0)]
num_borrowers [Row(sum(num_borrowers_match)=0)]
dti [Row(sum(dti_match)=0)]
borrower_credit_score [Row(sum(borrower_credit_score_match)=0)]
first_home_buyer [Row(sum(first_home_buyer_match)=331)]
loan_purpose [Row(sum(loan_purpose_match)=0)]
property_type [Row(sum(property_type_match)=49)]
num_units [Row(sum(num_units_match)=0)]
occupancy_status [Row(sum(occupancy_status_match)=0)]
property_state [Row(sum(property_state_match)=0)]
zip [Row(sum(zip_match)=0)]
mortgage_insurance_percent [Row(sum(mortgage_insurance_percent_match)=0)]
product_type [Row(sum(p

In [90]:
acq.select(['loan_id','mortgage_insurance_percent','mortgage_insurance_percent_old']).where('mortgage_insurance_percent_match = 1').show()

+-------+--------------------------+------------------------------+
|loan_id|mortgage_insurance_percent|mortgage_insurance_percent_old|
+-------+--------------------------+------------------------------+
+-------+--------------------------+------------------------------+

